In [1]:
import os
from pathlib import Path
from PIL import Image


import torch
from torchvision import transforms
from torch.autograd import Variable

from tqdm import tqdm_notebook as tqdm

import sys
sys.path.insert(0, './model')

import model.classifier as classifier

In [2]:
# Define a global transformer to appropriately scale images and subsequently convert them to a Tensor.
img_size = 224
loader = transforms.Compose([
  transforms.Resize(img_size),
  transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 

def load_image(filename):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
    image_tensor = loader(image).float()
    image_var = Variable(image_tensor).unsqueeze(0)
    return image_var

load_image('data/train_data/Drew_Barrymore/aligned_vgg_1c503cf0fb1b3178ca39f7c9919b34ad.jpg')

tensor([[[[0.2824, 0.2863, 0.2902,  ..., 0.9686, 0.9765, 0.9922],
          [0.2863, 0.2941, 0.3020,  ..., 0.9569, 0.9686, 0.9922],
          [0.2902, 0.3059, 0.3255,  ..., 0.9373, 0.9569, 0.9843],
          ...,
          [0.4471, 0.4902, 0.5569,  ..., 0.7412, 0.6980, 0.6863],
          [0.4275, 0.4627, 0.5176,  ..., 0.7176, 0.6863, 0.6784],
          [0.4392, 0.4588, 0.4980,  ..., 0.7020, 0.6745, 0.6745]],

         [[0.1608, 0.1647, 0.1686,  ..., 0.8392, 0.8588, 0.8745],
          [0.1647, 0.1725, 0.1804,  ..., 0.8275, 0.8510, 0.8745],
          [0.1686, 0.1843, 0.2000,  ..., 0.8078, 0.8392, 0.8667],
          ...,
          [0.2353, 0.2784, 0.3451,  ..., 0.4784, 0.4314, 0.4196],
          [0.2196, 0.2510, 0.3059,  ..., 0.4549, 0.4196, 0.4118],
          [0.2314, 0.2549, 0.2863,  ..., 0.4392, 0.4078, 0.4078]],

         [[0.0784, 0.0824, 0.0863,  ..., 0.7098, 0.7255, 0.7412],
          [0.0824, 0.0902, 0.0980,  ..., 0.6980, 0.7176, 0.7412],
          [0.0863, 0.1020, 0.1176,  ..., 0

In [3]:
train_data_folder = './data/train_data'

class_to_name = [os.path.basename(f.path) for f in os.scandir(train_data_folder) if f.is_dir()]

train_id_to_file = {i : path 
                    for (i,path) in enumerate(Path(train_data_folder).glob("*/*.jpg"))}

train_id_to_class = {i : class_to_name.index(os.path.basename(os.path.dirname(path)))
                     for (i,path) in enumerate(Path(train_data_folder).glob("*/*.jpg"))}

train_ids = list(train_id_to_file.keys())

print("Classes:")
print(class_to_name)

print("\nTraining Set Size: %s" % len(train_ids))

print("\nSample Image:")
print(train_id_to_file[1514])
print(class_to_name[train_id_to_class[1514]])

load_image(train_id_to_file[1514])

Classes:
['Zac_Efron', 'Cristiano_Ronaldo', 'Aaron_Eckhart', 'Clive_Owen', 'Alyssa_Milano', 'Christina_Ricci', 'Nicole_Richie', 'Brad_Pitt', 'Julia_Roberts', 'Drew_Barrymore']

Training Set Size: 1515

Sample Image:
data/train_data/Drew_Barrymore/aligned_vgg_1c503cf0fb1b3178ca39f7c9919b34ad.jpg
Drew_Barrymore


tensor([[[[0.2824, 0.2863, 0.2902,  ..., 0.9686, 0.9765, 0.9922],
          [0.2863, 0.2941, 0.3020,  ..., 0.9569, 0.9686, 0.9922],
          [0.2902, 0.3059, 0.3255,  ..., 0.9373, 0.9569, 0.9843],
          ...,
          [0.4471, 0.4902, 0.5569,  ..., 0.7412, 0.6980, 0.6863],
          [0.4275, 0.4627, 0.5176,  ..., 0.7176, 0.6863, 0.6784],
          [0.4392, 0.4588, 0.4980,  ..., 0.7020, 0.6745, 0.6745]],

         [[0.1608, 0.1647, 0.1686,  ..., 0.8392, 0.8588, 0.8745],
          [0.1647, 0.1725, 0.1804,  ..., 0.8275, 0.8510, 0.8745],
          [0.1686, 0.1843, 0.2000,  ..., 0.8078, 0.8392, 0.8667],
          ...,
          [0.2353, 0.2784, 0.3451,  ..., 0.4784, 0.4314, 0.4196],
          [0.2196, 0.2510, 0.3059,  ..., 0.4549, 0.4196, 0.4118],
          [0.2314, 0.2549, 0.2863,  ..., 0.4392, 0.4078, 0.4078]],

         [[0.0784, 0.0824, 0.0863,  ..., 0.7098, 0.7255, 0.7412],
          [0.0824, 0.0902, 0.0980,  ..., 0.6980, 0.7176, 0.7412],
          [0.0863, 0.1020, 0.1176,  ..., 0

In [4]:
model = classifier.FaceClassifier()
print(model)

FaceClassifier(
  (model): Vgg_face_dag(
    (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu1_1): ReLU(inplace)
    (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu1_2): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu2_1): ReLU(inplace)
    (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu2_2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_1): ReLU(inplace)
    (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_2): ReLU(inplace)
    (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
    (relu3_3):

In [ ]:
# TODO - training

images = torch.zeros((len(train_ids),3,img_size,img_size))
labels = torch.zeros(len(train_ids)).long()

for i,train_id in enumerate(tqdm(train_ids)):
    # Prepare the image tensors
    images[i] = load_image(train_id_to_file[train_id])
    
    # Prepare the labels
    labels[i] = train_id_to_class[train_id]

X = images
y = labels


model.tune(X, y)

torch.save(model.state_dict(), 'model/tuned_classifier.pth')

[1,     5] loss: 2.279
[1,    10] loss: 2.224
[1,    15] loss: 2.166
